**Amaç: Her bir eve ait özelliklerin ve ev fiyatlarının bulunduğu veriseti kullanılarak, farklı tipteki
evlerin fiyatlarına ilişkin bir makine öğrenmesi projesi gerçekleştirilmek istenmektedir.**

> Gerekli kütüphanelerin import edilmesi

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV, cross_val_score

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

> Test ve train seti olarak ayrılan veri setini birleştirme

In [ ]:
train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
df = train.append(test).reset_index(drop=True)
df.head()

> Veri hakkında bilgi edinme

In [ ]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [ ]:
check_df(df)

> Boş değer kontrolü

In [ ]:
df.isnull().sum()

> Toplam boş değer sayısı

In [ ]:
df.isnull().sum().sum()

> MSZoning: İmar sınıflandırmasını veriyor.
> MSSubClass : Evlerin stili hakkında bilgi veriyor.
> MSZoning'deki boş değerleri evlerin stili hakkındaki bilgiyi veren değişkene göre gruplandırıp kategorik değişken olduğundan mode'u ile dolduruyoruz. Çünkü evlerin stiliyle kullanılma amacı arasında bir ilişki vardır bu mantığa uygun.

In [ ]:
df["MSZoning"].describe()
df['MSZoning'] = df.groupby('MSSubClass')['MSZoning'].apply(lambda x: x.fillna(x.mode()[0]))
df['Exterior1st'] = df['Exterior1st'].fillna(df['Exterior1st'].mode()[0])
df['Exterior2nd'] = df['Exterior2nd'].fillna(df['Exterior2nd'].mode()[0])

In [ ]:
df["TotalBsmtSF"].value_counts()

> "TotalBsmtSF" : Toplam bodrum yüzölçümünü veriyor.Buradaki nan değerler o evde bodrum olmadığını gösteriyor eksik gözlem değil, böyle birçok değişken olduğundan bir liste oluşturulur tüm columnlar incelendikten sonra hepsi for döngüsüyle nan'dan 0'a çevrilir. 

In [ ]:
df[df["TotalBsmtSF"]==0]
nan_to_zero=[]
nan_to_zero.append("TotalBsmtSF")

In [ ]:
nan_to_zero.append("LotFrontage")

In [ ]:
nan_to_zero.append("Alley")

In [ ]:
def twice_col(df,col1,col2):
    n=df[col1].unique().tolist()
    l=len(n)
    n1=df[col2].unique().tolist()
    l1=len(n1)
    for i in n:
        for j in range(l):
            for z in n1:
                for t in range(l1):
                    df.loc[(df[col1] == n[j]) & (df[col2] == n1[t]), str(col1) + "_" + str(col2)]=str(n[j]) + str(n1[t])

In [ ]:
twice_col(df,"Heating","Electrical")

In [ ]:
df[df["Utilities"].isna()]['Heating_Electrical']
df.groupby(['Heating_Electrical'])["Utilities"].agg(pd.Series.mode)
df["Utilities"].value_counts()
df["Utilities"]=df["Utilities"].fillna("AllPub")
df["Utilities"].isnull().sum()

> "PoolArea" : Havuzun alan ölçüsü. 
> "PoolQC" : Havuzun kalitesi hakkında bilgi.
> Bu değişkenlerdeki boş değerler de o evde havuz olmadığını gösteriyor aynı şekilde listeye eklenir.

In [ ]:
df[['PoolArea',"PoolQC",]].head()
nan_to_zero.append("PoolQC")
nan_to_zero.append('PoolArea')

In [ ]:
nan_to_zero.append("MiscFeature")
nan_to_zero.append("Fence")
nan_to_zero.append("FireplaceQu")

In [ ]:
nan_to_zero.append("GarageQual")
nan_to_zero.append("GarageFinish")

In [ ]:
df[df["GarageYrBlt"].isnull()][["GarageYrBlt","GarageType"]]
df.loc[df["GarageYrBlt"].isnull(),"GarageType"]="None"

In [ ]:
nan_to_zero.append("GarageYrBlt")
df["BsmtExposure"].value_counts()
df["BsmtExposure"].isnull().sum()
nan_to_zero.append("BsmtExposure")

In [ ]:
nan_to_zero.append("BsmtQual")
nan_to_zero.append("BsmtFinType1")
nan_to_zero.append("BsmtFinType2")

In [ ]:
df.loc[df["MasVnrType"].isnull(),"MasVnrArea"].notnull()
df.loc[df["MasVnrArea"].isnull(),"MasVnrType"]
df.loc[df["MasVnrArea"].isnull(),"MasVnrType"]="None"
df[df["MasVnrType"]=="None"]["MasVnrArea"]

In [ ]:
nan_to_zero.append("MasVnrArea")
nan_to_zero.append("GarageCond")
nan_to_zero.append("BsmtFullBath")
nan_to_zero.append("BsmtHalfBath")

In [ ]:
for i in nan_to_zero:
    if i != "SalePrice":
        print(df[i].head())

> Listede bulunan değişkenler hakkında bilgi sahibi olunması ve type bilgisine ulaşılması için head atılır

> Ve tipi object olan columnlardaki boş değerler none ile object olmayanların boş değerleri ise 0 ile doldurulur.

In [ ]:
for col in nan_to_zero:
    if df[col].dtype == "O":
        df.loc[df[col].isnull(), col] = "None"
    else:
        df.loc[df[col].isnull(), col] = 0

In [ ]:
df.isnull().sum()

> Test ve train seti birleştirildiğinden test setindeki sales değerleri boş gelecektir onun dışında birkaç columnda daha boş değer olduğu görünmekte onlar da dönüştürülür.

In [ ]:
na_cols = [col for col in df.columns if df[col].isnull().sum() > 0]

In [ ]:
for i in na_cols:
    if i != "SalePrice":
        print(df[i].head())

In [ ]:
for col in na_cols:
    if col!="SalePrice":
        if df[col].dtype=="O":
            df.loc[df[col].isnull(), col] = "None"
        else:
            df.loc[df[col].isnull(),col]=0

In [ ]:
df['YearBuilt'].describe()
df["GarageYrBlt"].describe()

In [ ]:
df.loc[df["GarageYrBlt"]==2207]

> Yıl değişkeninde gürültülü bir veri var bu gözlemin satış yılına bakıldığında 2007 yazdığı görülmektedir çok büyük ihtimal garaj yapım yılı da 2007 olacaktır, dönüştürülür.

In [ ]:
df.loc[df["GarageYrBlt"]==2207,"GarageYrBlt"]=2007

> Aykırı değerler baskılanır.

In [ ]:
sns.boxplot(df["SalePrice"])
plt.show()

In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

In [ ]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

In [ ]:
def outlier_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [ ]:
for i in num_cols:
    outlier_thresholds(df, i, q1=0.1, q3=0.90)

In [ ]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [ ]:
for i in num_cols:
    replace_with_thresholds(df,i)

In [ ]:
sns.boxplot(df["SalePrice"])
plt.show()

> Kaliteli özellikler üretebilmek için columnların saleprice bağımlı değişkeniyle olan korelasyonuna bakılır.

In [ ]:
def find_correlation(dataframe, numeric_cols, corr_limit=0.60):
    high_correlations = []
    low_correlations = []
    for col in numeric_cols:
        if col == "SalePrice":
            pass
        else:
            correlation = dataframe[[col, "SalePrice"]].corr().loc[col, "SalePrice"]
            print(col, correlation)
            if abs(correlation) > corr_limit:
                high_correlations.append(col + ": " + str(correlation))
            else:
                low_correlations.append(col + ": " + str(correlation))
    return low_correlations, high_correlations

In [ ]:
find_correlation(df, num_cols)

In [ ]:
df["newww"]=df["TotalBsmtSF"]*df["OverallQual"]
df["NEW_area"]=df['1stFlrSF']+df["2ndFlrSF"]

> Gözlemlerdeki kategorik bilgi taşıyan veriler sayısal değerlere dönüştürülür böylelikle aralarında ordinallik sağlanır.Bu işlem birçok column'a uygulanacağından aynı işlemleri tekrar etmemek adına bir fonksiyon oluşturulur.

In [ ]:
def new_sorting(df,col,old,new):
    maps={}
    n=len(old)
    for i in range(n):
        maps.setdefault(old[i],new[i])
    df[col] = df[col].map(maps).astype(int)

In [ ]:
df["Functional"].head()
df["Functional"].unique()
df.groupby(["Functional"])[["SalePrice"]].mean().sort_values(by="SalePrice",ascending=False)
df["Functional"].value_counts()
new_sorting(df,"Functional",['Typ', 'Min1', 'Maj1', 'Min2', 'Mod', 'Maj2', 'Sev', 'None'],[6,5,5,4,5,5,3,1])

In [ ]:
df["new_sey"]=df["YrSold"]-df["YearBuilt"]
df["new_sey2"]=df["YrSold"]-df["YearRemodAdd"]

df["MoSold"]=[int(i) for i in df["MoSold"]]
df["MoSold"].describe()

In [ ]:
for i in range(df.shape[0]):
    if df.loc[i,"MoSold"] in [12,1,2]:
        df.loc[i,"season"]=1
    elif df.loc[i,"MoSold"] in [3,4,5]:
        df.loc[i, "season"] = 2
    elif df.loc[i,"MoSold"] in [6,7,8]:
        df.loc[i, "season"] = 3
    else:
        df.loc[i, "season"] = 4

In [ ]:
df["new"]=df["TotRmsAbvGrd"]+df["FullBath"]+df["HalfBath"]+df["BsmtFullBath"]+df["BsmtHalfBath"]

> Aynı şekilde bazı columnlarda ise çeyreklik değerlerine göre bir sıralama yapılmak istendiğinden ve bu işlem birçok kez tekrarlanacağından böyle bir fonksiyon oluşturulur.

In [ ]:
def generate(df,c,a,b,t,y,y1,y2):
    df.loc[(df[c] < a), t] = y
    df.loc[(df[c] >= a) & (df[c] < b), t] = y1
    df.loc[(df[c] >= b), t] = y2
    return df[t]

In [ ]:
df["new2"]=df["BsmtUnfSF"]/df["TotalBsmtSF"]
df["new2"].describe()
generate(df,"new2",0.21,0.47,"new2_new",3,2,1)

In [ ]:
df["BsmtFinType1"].unique()

In [ ]:
new_sorting(df,"BsmtFinType1",['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', 'None', 'LwQ'],[5,4,1,3,3,0,2])
new_sorting(df,"BsmtFinType2",['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', 'None', 'LwQ'],[5,4,1,3,3,0,2])
df["new3"]=df["BsmtFinType1"]+df["BsmtFinType2"]

> GrLivArea : Oturma alanı metre karesi. (yaşam alanı)
LotArea: Eve ait arsanın alan ölçümünü veriyor.
Bu özellik de toplam yaşam alanının toplam evin alanına(arsa) oranını veriyor.

In [ ]:
df["new4"]=df["GrLivArea"]/df["LotArea"]

In [ ]:
df["new5"]=df["YrSold"]-df["GarageYrBlt"]
df["new7"]=df["GarageCars"]*df["GarageArea"]
df["new8"]=df['OverallQual']*df['OverallCond']

In [ ]:
df["new9"]=df["TotRmsAbvGrd"]+df["HalfBath"]+df['FullBath']

In [ ]:
df["new10"]=(df['BsmtFinSF1']+df["BsmtFinSF2"])/df["TotalBsmtSF"]

In [ ]:
generate(df,"new10",0.41,0.71,"new10_new",1,2,3)

In [ ]:
df["new11"]=df["KitchenAbvGr"]+df["BedroomAbvGr"]

In [ ]:
new_sorting(df,"PavedDrive",["Y","P","N"],[3,2,1])
df["PavedDrive"]
df["new_quality"]=df['WoodDeckSF']+df['OpenPorchSF']+df['EnclosedPorch']+df['3SsnPorch']+df['ScreenPorch']+df['PoolArea']

In [ ]:
new_sorting(df,"LotShape",["Reg","IR1","IR2","IR3"],[4,3,2,1])
new_sorting(df,"LandContour",["Lvl","Bnk","HLS","Low"],[4,3,2,1])
df['LotConfig'].unique()
new_sorting(df,"LotConfig",['Inside','Corner','CulDSac','FR2','FR3'],[2,3,1,4,5])

In [ ]:
df['BldgType'].unique()
new_sorting(df,'BldgType',['1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'],[1,2,2,3,3])

In [ ]:
df.groupby("HouseStyle")[["SalePrice"]].mean().sort_values(by="SalePrice",ascending=False)
df["HouseStyle"].unique()

In [ ]:
new_sorting(df,"HouseStyle",['2Story', '1Story', '1.5Fin', '1.5Unf', 'SFoyer', 'SLvl', '2.5Unf',
       '2.5Fin'],[7,6,3,1,2,5,4,8])
df['GarageType'].unique()
new_sorting(df,'GarageType',['Attchd', 'Detchd', 'BuiltIn', 'CarPort', 'None', 'Basment',
       '2Types'],[3,5,2,1,0,4,5])

In [ ]:
df["GarageFinish"].unique()

In [ ]:
new_sorting(df,"GarageFinish",['RFn', 'Unf', 'Fin', 'None']
,[2,1,3,0])

In [ ]:
df.loc[(df[df["KitchenQual"]=="None"].index),"KitchenQual"]="Po"
df.loc[(df[df['BsmtQual']=="None"].index),'BsmtQual']="Po"
df.loc[(df[df["BsmtExposure"]=="None"].index),"BsmtExposure"]="Po"
df.loc[(df[df["PoolQC"]=="None"].index),"PoolQC"]="Po"

In [ ]:
old=["Ex","Gd","TA","Fa","Po","NA","No","Av","Mn",'None',0]
new=[5,4,3,2,1,0,0,2,1,0,0]
for col in ["ExterCond","HeatingQC","KitchenQual","PoolQC","FireplaceQu",'BsmtCond','BsmtQual',"BsmtExposure","ExterQual","GarageQual","GarageCond"]:
    new_sorting(df,col,old,new)

In [ ]:
df["BsmtFinType1"].value_counts()
df.groupby(["BsmtFinType1"])["SalePrice"].mean().sort_values(ascending=False)
df.groupby(["BsmtFinType2"])["SalePrice"].mean().sort_values(ascending=False)

> Bir evde havuz varsa 1 döndürür yoksa 0 döndürür
Aynı şekilde garaj varsa 1 yoksa 0 döndürür.
Diğer feature'lar da aynı mantıkla oluşturulur.

In [ ]:
df['PoolArea']=[1 if i>0 else 0 for i in df['PoolArea']]
df['GarageCars']=[1 if i>0 else 0 for i in df['GarageCars']]
df['Fireplaces']=[1 if i>0 else 0 for i in df['Fireplaces']]
df["CentralAir"]=[1 if i=="Y" else 0 for i in df["CentralAir"]]
df["MiscFeature"]=[0 if i=="None" else 1 for i in df["MiscFeature"]]

In [ ]:
df.groupby("MSZoning")[["SalePrice"]].mean()
df["MSZoning"].unique()
new_sorting(df,"MSZoning",["C (all)",'RL','RM','FV','RH'],[1,5,3,4,2])
new_sorting(df,"Alley",["Grvl","Pave",'None'],[1,2,0])

In [ ]:
df["Utilities"].unique()
df["Utilities"]=[2 if i=='AllPub' else 1 for i in df["Utilities"]]

In [ ]:
df["LandSlope"].unique()
new_sorting(df,"LandSlope",["Gtl","Mod",'Sev'],[3,2,1])
df.groupby(["Neighborhood"])["SalePrice"].mean().sort_values(ascending=False)
df["Neighborhood"].unique()
old=['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst',
       'NWAmes', 'OldTown', 'BrkSide', 'Sawyer', 'NridgHt', 'NAmes',
       'SawyerW', 'IDOTRR', 'MeadowV', 'Edwards', 'Timber', 'Gilbert',
       'StoneBr', 'ClearCr', 'NPkVill', 'Blmngtn', 'BrDale', 'SWISU',
       'Blueste']
new=[5,9,6,12,4,8,5,2,2,3,11,4,5,1,1,2,9,5,10,7,4,5,1,4,3]
new_sorting(df,"Neighborhood",old,new)

In [ ]:
df["newfeac"]=df['KitchenAbvGr']*df['KitchenQual']
df["newfeac2"]=df['Fireplaces']*df['FireplaceQu']
df["Fence"].unique()
new_sorting(df,"Fence",['None', 'MnPrv', 'GdWo', 'GdPrv', 'MnWw'],[0,1,2,2,1])

In [ ]:
df["Street"]=[0 if i == "Gravel" else 1 for i in df["Street"]]

In [ ]:
new_sorting(df,"Heating",['GasA', 'GasW', 'Grav', 'Wall', 'OthW', 'Floor'],[3,3,1,2,3,1])

> Roofstyle değişkeni çatıların özelliklerini belirtmektedir çatı görsellerine ve saleprice'a göre ortalamalara bakıldığında bu şekilde bir sıralama elde edilir.

In [ ]:
df["RoofStyle"].unique()
new_sorting(df,"RoofStyle",['Gable', 'Hip', 'Gambrel', 'Mansard', 'Flat', 'Shed']
,[4,2,2,1,3,1])

In [ ]:
df.groupby("RoofMatl")[["SalePrice"]].mean().sort_values(by="SalePrice",ascending=False)
df["RoofMatl"].unique()
new_sorting(df,"RoofMatl",['CompShg', 'WdShngl', 'Metal', 'WdShake', 'Membran', 'Tar&Grv',
       'Roll', 'ClyTile'],[3,6,4,5,5,4,1,2])

In [ ]:
df["Electrical"].unique()
df.groupby("Electrical")[["SalePrice"]].mean().sort_values(by="SalePrice",ascending=False)

In [ ]:
new_sorting(df,"Electrical",['SBrkr', 'FuseF', 'FuseA', 'FuseP', 'Mix', 'None'],[5,2,3,1,0,4])
df.groupby("SaleType")[["SalePrice"]].mean().sort_values(by="SalePrice",ascending=False)
df["SaleType"].value_counts()
df["SaleType"].unique()

In [ ]:
new_sorting(df,"SaleType",['WD', 'New', 'COD', 'ConLD', 'ConLI', 'CWD', 'ConLw', 'Con', 'Oth',
       'None'],[5,8,4,2,6,7,3,9,1,0])

In [ ]:
df.groupby("SaleCondition")[["SalePrice"]].mean().sort_values(by="SalePrice",ascending=False)
df["SaleCondition"].value_counts()
df["SaleCondition"].unique()
df.head()
new_sorting(df,"SaleCondition",['Normal', 'Abnorml', 'Partial', 'AdjLand', 'Alloca', 'Family'],[4,2,5,1,3,2])

In [ ]:
df.groupby("Foundation")[["SalePrice"]].mean().sort_values(by="SalePrice",ascending=False)
df["Foundation"].unique()
new_sorting(df,"Foundation",['PConc', 'CBlock', 'BrkTil', 'Wood', 'Slab', 'Stone'],[5,2,1,4,0,3])

df["MasVnrType"].unique()
df.groupby("MasVnrType")[["SalePrice"]].mean().sort_values(by="SalePrice",ascending=False)
# ['BrkFace', 'None', 'Stone', 'BrkCmn']
new_sorting(df,"MasVnrType",['BrkFace', 'None', 'Stone', 'BrkCmn'],[3,2,4,1])
df["Condition1"].unique()
new_sorting(df,"Condition1",['Norm', 'Feedr', 'PosN', 'Artery', 'RRAe', 'RRNn', 'RRAn', 'PosA',
       'RRNe'],[2,1,3,1,1,3,2,3,2])
new_sorting(df,"Condition2",['Norm', 'Feedr', 'PosN', 'Artery', 'RRAe', 'RRNn', 'RRAn', 'PosA',
       'RRNe'],[2,1,3,1,1,3,2,3,2])

In [ ]:
df["Exterior1st"].value_counts()
df["Exterior1st"].unique()
df.groupby("Exterior1st")[["SalePrice"]].mean().sort_values(by="SalePrice",ascending=False)

In [ ]:
new_sorting(df,"Exterior1st",['VinylSd', 'MetalSd', 'Wd Sdng', 'HdBoard', 'BrkFace', 'WdShing',
       'CemntBd', 'Plywood', 'AsbShng', 'Stucco', 'BrkComm', 'AsphShn',
       'Stone', 'ImStucc', 'CBlock'],[7,3,3,4,6,3,8,5,2,4,1,2,9,10,2])

In [ ]:
df.loc[(df["Exterior2nd"]=="Other"),"Exterior2nd"]='ImStucc'
new_sorting(df,"Exterior2nd",['VinylSd', 'MetalSd', 'Wd Shng', 'HdBoard', 'Plywood', 'Wd Sdng',
       'CmentBd', 'BrkFace', 'Stucco', 'AsbShng', 'Brk Cmn', 'ImStucc',
       'AsphShn', 'Stone', 'CBlock'],[7,3,3,4,5,3,8,6,4,2,1,10,2,9,2])

In [ ]:
df.head()

In [ ]:
check_df(df)

> Gereksiz columnlar silinir.

In [ ]:
useless_cols = [col for col in cat_cols if df[col].nunique() == 1 or
                (df[col].nunique() == 2 and (df[col].value_counts() / len(df) <= 0.01).any(axis=None))]

cat_cols = [col for col in cat_cols if col not in useless_cols]


for col in useless_cols:
    df.drop(col, axis=1, inplace=True)

In [ ]:
df.drop("Heating_Electrical",axis=1,inplace=True)

In [ ]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)
cat_cols = cat_cols + cat_but_car

> Kategorik columnlar modele girebilmek için encoding işlemine tabi tutulur.

In [ ]:
def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

In [ ]:
df = one_hot_encoder(df, cat_cols, drop_first=True)

cat_cols, num_cols, cat_but_car = grab_col_names(df)

useless_cols_new = [col for col in cat_cols if (df[col].value_counts() / len(df) <= 0.01).any(axis=None)]

In [ ]:
df[useless_cols_new].head()

 > Model kurulur.

In [ ]:
train_df = df[df['SalePrice'].notnull()]
test_df = df[df['SalePrice'].isnull()].drop("SalePrice", axis=1)
y = np.log1p(train_df['SalePrice'])
X = train_df.drop(["Id", "SalePrice"], axis=1)

In [ ]:
models = [('LR', LinearRegression()),
          ("Ridge", Ridge()),
          ("Lasso", Lasso()),
          ("ElasticNet", ElasticNet()),
          ('KNN', KNeighborsRegressor()),
          ('CART', DecisionTreeRegressor()),
          ('RF', RandomForestRegressor()),
          ('SVR', SVR()),
          ('GBM', GradientBoostingRegressor()),
          ("XGBoost", XGBRegressor(objective='reg:squarederror')),
          ("LightGBM", LGBMRegressor()),
          ("CatBoost", CatBoostRegressor(verbose=False))]

In [ ]:
for name, regressor in models:
    rmse = np.mean(np.sqrt(-cross_val_score(regressor, X, y, cv=5, scoring="neg_mean_squared_error")))
    print(f"RMSE: {round(rmse, 4)} ({name}) ")

> LightGBM ile hiperparametre optimizasyonu yapılır örneğin learning rate parametresine girilen birçok değer diğer iki parametredeki değerlerin her biriyle işleme sokulur bunu sağlayan GridSearchCV'dir ve en düşük hatayı veren parametre değerleri lgbm_best_params olarak atanır. Model en iyi parametrelerle tekrar kurulur ve rmse(hata değeri) tekrar hesaplanır.

In [ ]:
lgbm_model = LGBMRegressor(random_state=46)

rmse = np.mean(np.sqrt(-cross_val_score(lgbm_model,
                                        X, y, cv=5, scoring="neg_mean_squared_error")))


lgbm_params = {"learning_rate": [0.01,0.02,0.03,0.05,0.1, 0.2],
               "n_estimators": [100,200,300,500,1000,1200,1500],
               "colsample_bytree": [0.2,0.3,0.5, 0.7,0.8,0.9,1]}


lgbm_gs_best = GridSearchCV(lgbm_model,
                            lgbm_params,
                            cv=3,
                            n_jobs=-1,
                            verbose=True).fit(X, y)


In [ ]:
final_model = lgbm_model.set_params(**lgbm_gs_best.best_params_).fit(X, y)

rmse = np.mean(np.sqrt(-cross_val_score(final_model, X, y, cv=5, scoring="neg_mean_squared_error")))

rmse

> Oluşturulan yeni özelliklerin önem düzeylerine plot_importance fonksiyonuyla erişilir

In [ ]:
def plot_importance(model, features, num=len(X), save=False):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features.columns})
    plt.figure(figsize=(10, 10))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')

In [ ]:
plot_importance(final_model, X, 20)

In [ ]:
def num_summary(dataframe, numerical_col, plot=False):
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[numerical_col].describe(quantiles).T)

In [ ]:
feature_imp = pd.DataFrame({'Value': final_model.feature_importances_, 'Feature': X.columns})

feature_imp[feature_imp["Value"] > 0].shape

feature_imp[feature_imp["Value"] < 1].shape


zero_imp_cols = feature_imp[feature_imp["Value"] < 1]["Feature"].values


> Bu işlem sonrasında önem düzeylerine bakılan özelliklerin önemlileriyle tekrar bir model kurulur. Bu şekilde hata azaltılmaya çalışılır.

In [ ]:
selected_cols = [col for col in X.columns if col not in zero_imp_cols]

In [ ]:
lgbm_model = LGBMRegressor(random_state=46)

lgbm_params = {"learning_rate": [0.01,0.02,0.03,0.05,0.1, 0.2],
               "n_estimators":  [100,200,300,500,1000,1200,1500],
               "colsample_bytree": [0.2,0.3,0.5, 0.7,0.8,0.9,1]}

lgbm_gs_best = GridSearchCV(lgbm_model,
                            lgbm_params,
                            cv=3,
                            n_jobs=-1,
                            verbose=True).fit(X[selected_cols], y)


final_model = lgbm_model.set_params(**lgbm_gs_best.best_params_).fit(X[selected_cols], y)

rmse = np.mean(np.sqrt(-cross_val_score(final_model, X[selected_cols], y, cv=5, scoring="neg_mean_squared_error")))

In [ ]:
rmse